In [17]:
#Importação da bibliotecas necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

#from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
import scipy.stats as stats
url = "https://raw.githubusercontent.com/amandacmelo/Projeto-Ciencia-de-Dados/main/dados_limpos.csv"
dados = pd.read_csv(url, sep=",")

#dados = pd.read_csv(url, on_bad_lines="skip", sep=";") #lida com linhas com algum espaço vazio
dados.columns



Index(['Ano', 'CDMunicipio', 'Populacao', 'PessoalOcupado',
       'PessoalAssalariado', 'VrSalarios', 'PIB', 'QtEmpresas',
       'AreaPlantada_h', 'AreaColhida_h', 'VlProducaoAgricola',
       'VlProducaoPecuaria', 'Area', 'Importacoes_US$', 'Exportacoes_US$',
       'Receitas_R$', 'Transferencias_correntes_R$',
       'Transferencias_capital_R$', 'NrNascimentos', 'NrObitosInfantis',
       'povoamento', 'UF', 'Municipio', 'Regiao', 'PIB_PerCapita',
       'Densidade_Populacional', 'Proporcao_PessoalAssalariado',
       'Proporcao_Area_Plantada'],
      dtype='object')

# 🟤 Regressão - Municípios com maior produção agrícola tendem a ter maior área desmatada?

Na entrega anterior do trabalho, foi utilizado regras de associação (Apriori) para explorar as relações entre desmatamento e variáveis agropecuárias. As regras extraídas indicaram uma associação positiva e significativa entre esses fatores. Por exemplo:

* Quando o desmatamento e a área plantada são altos, há uma probabilidade de ~77% de que a produção agrícola também seja alta.

Esses achados sugerem que o aumento da produção agrícola e da área plantada está correlacionado com maior desmatamento, o que motivou a aplicação de uma análise de regressão nesta etapa, com o objetivo de  avaliar essa relação de forma quantitativa.

## Preparação dos dados
Nesta etapa, os dados de produção agrícola foram combinados com os dados de desmatamento por município e ano. Para isso, foi necessário padronizar os nomes das colunas de identificação e realizar a mesclagem entre os conjuntos.

Após a junção, foram selecionadas apenas as variáveis relevantes para a análise: produção agrícola, área desmatada, área plantada, área colhida e produção pecuária.


In [146]:

url1 = "https://raw.githubusercontent.com/amandacmelo/Projeto-Ciencia-de-Dados/refs/heads/main/Dados%20sobre%20Desmatamento/area_desmatada_municipio.csv"
dados_desmatamento = pd.read_csv(url1, sep=",")

# Renomear colunas para facilitar a mesclagem
dados = dados.rename(columns={'Ano': 'ano', 'CDMunicipio': 'id_municipio'})

# Mesclar os datasets com base nas colunas 'ano' e 'id_municipio'
df = pd.merge(dados, dados_desmatamento, on=['ano', 'id_municipio'], how='inner')

# Selecionar apenas as colunas relevantes
df = df[['VlProducaoAgricola', 'desmatado', 'AreaPlantada_h', 'VlProducaoPecuaria']].copy()

## 📈 Regressão Linear Simples: Área Plantada e Desmatamento
Este código aplica uma regressão linear simples para investigar se existe uma relação entre a área plantada e a área desmatada nos municípios. A ideia é verificar se, à medida que aumenta a área utilizada para plantio, também ocorre um aumento proporcional no desmatamento.

O modelo estima uma equação de reta que melhor se ajusta aos dados observados, permitindo medir a força e a direção dessa relação. Além disso, são calculadas métricas como o R², que indica o quanto da variação no desmatamento pode ser explicada pela área plantada, e o Erro Médio Absoluto (MAE), que mostra o quão próximas as previsões do modelo estão dos valores reais.

Essa abordagem ajuda a responder se áreas mais dedicadas à agricultura tendem, de fato, a apresentar maior desmatamento — e com qual intensidade.

In [147]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Variáveis independentes (X) e dependentes (Y)
X = df['AreaPlantada_h'].values.reshape(-1, 1)  # X deve estar em formato 2D
Y = df['desmatado'].values  # Y pode estar em formato 1D

# Criar e ajustar o modelo de regressão linear
lm = LinearRegression()
lm.fit(X, Y)

# Coeficiente (beta), intercepto e R²
print('Coeficiente estimado (β):', lm.coef_[0])
print('Intercepto:', lm.intercept_)
print('R² (score):', lm.score(X, Y))

# Previsões e Erro Médio Absoluto
Y_pred = lm.predict(X)
mae = mean_absolute_error(Y, Y_pred)
print('Erro Médio Absoluto (MAE):', mae)


Coeficiente estimado (β): 0.27304162419860073
Intercepto: 373.271883024796
R² (score): 0.10175792569754205
Erro Médio Absoluto (MAE): 398.76426000958423


### Conclusão
A regressão linear confirmou parcialmente a ideia de que a expansão agrícola está associada ao desmatamento. O coeficiente β = 0,273 indica uma relação positiva entre área plantada e área desmatada, alinhando-se aos achados anteriores das regras de associação.

No entanto, o R² de apenas 10,18% mostra que a área plantada explica pouco da variação no desmatamento. O erro médio absoluto de 398,76 reforça as limitações do modelo.

Assim, embora exista associação, a relação é mais complexa e depende de outros fatores não considerados, como políticas públicas, tipo de cultura e contexto regional. Modelos mais completos são necessários para entender melhor o fenômeno.

## 📈 Regressão Linear Múltipla: Váriáveis Agropecuárias e o Desmatamento

Levando em consideração a limitação levantada anteriormente, foi realizada uma regressão linear múltipla incluindo as variáveis produção agrícola e pecuária, além da área plantada, para verificar se a adição dessas variáveis auxilia no desempenho do modelo para a explicação da variação da área desmatada.

Essa abordagem busca compreender se o desmatamento está mais fortemente relacionado ao conjunto de fatores ligados à atividade agropecuárias, em comparação ao modelo anterior, que utilizava apenas uma variável independente.

In [148]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Definindo as variáveis independentes (X) e a variável dependente (Y)
X = df[['VlProducaoAgricola', 'AreaPlantada_h', 'VlProducaoPecuaria']]
Y = df['desmatado']

# Criando e treinando o modelo de Regressão Linear
modelo = LinearRegression()
modelo.fit(X, Y)

# Coeficientes e intercepto do modelo
print('Coeficientes:')
print(pd.DataFrame(modelo.coef_, X.columns, columns=['Coeficiente']))
print('\nIntercepto:', modelo.intercept_)

# R² - Coeficiente de Determinação
r2 = modelo.score(X, Y)
print('\nR² (Score):', r2)

# Fazendo previsões com os dados de entrada
Y_pred = modelo.predict(X)

# Calculando o Erro Médio Absoluto (MAE)
mae = mean_absolute_error(Y, Y_pred)
print('Erro Médio Absoluto (MAE):', mae)


Coeficientes:
                    Coeficiente
VlProducaoAgricola     0.048980
AreaPlantada_h         0.227813
VlProducaoPecuaria     0.071929

Intercepto: 312.1512204393604

R² (Score): 0.12352226214911755
Erro Médio Absoluto (MAE): 390.4037677003679


### Conclusão
O modelo múltiplo, incorporando valor da produção, área plantada e área colhida, apresentou R² de 12.35% e MAE de 390.40, representando uma melhoria em relação ao modelo simples (R² = 10.18% e MAE = 398,76). Os coeficientes positivos das três variáveis (0.050 para valor da produção, 0.228 para área plantada e 0.072 para área colhida) confirmam que todos os aspectos da atividade agropecuárias contribuem para o desmatamento.

Entretanto, mesmo com a inclusão de múltiplas variáveis agropecuárias, o modelo ainda explica apenas cerca de 12% da variação no desmatamento, evidenciando a alta complexidade desta variável. Este baixo poder explicativo revela que aproximadamente 88% da variação no desmatamento é influenciada por fatores não capturados no modelo, como políticas ambientais, fiscalização, características geográficas, pressões econômicas locais e outros determinantes socioambientais.

Os resultados validam os achados das regras de associação anteriores, confirmando estatisticamente a relação positiva entre atividade agrícola e desmatamento, mas destacam que o desmatamento é um fenômeno multifatorial que transcende as variáveis puramente agropecuárias analisadas.